In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/home/dom/Documents/MPhys/TheGrandTour/wine_data.txt", sep="\t");
data = np.array(df);
data = np.delete(data, 0, 0)
data = data.astype(float)
data = np.swapaxes(data,0,1)


classification = data[13]
data = np.delete(data, 13, axis=0)
# Normalizes the data        
for i in range(0, np.shape(data)[0]):
    data[i,:] = (data[i,:] / np.ndarray.max(data[i,:])) * 2 - 1


In [4]:
stepSize = 0.01
nSteps = 10000

def getAlpha(d):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.
    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d
    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane
    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane
    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def GrandTour(data, nSteps):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    tData = np.zeros((nSteps, d, nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps, d, d))
    Alpha = getAlpha(d)

    
    for t in range(0, nSteps):
        
        
        alpha = Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        tData[t,:,:] = a
        tBeta[t,:,:] = b
        
    return tData, tBeta


tData, tBeta = GrandTour(data, nSteps)


In [6]:
targetData = np.zeros((len(tData[0][0]), 3))
for counter, i in enumerate(classification):
    targetData[counter][int(i-1)] = 1
targetData

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [19]:
x = torch.from_numpy(data)
y = torch.from_numpy(targetData)
print(x.size())
print(y.size())
#x.transpose_(0, 1)
print(x.size())
print(y.size())

torch.Size([13, 177])
torch.Size([177, 3])
torch.Size([13, 177])
torch.Size([177, 3])


In [27]:
print(x.size())
x.transpose_(0, 1)
x.size()

torch.Size([13, 177])


torch.Size([177, 13])

In [60]:
#print(x[0])
#print(x[1])
print(x.size())
print(y.size())

torch.Size([177, 13])
torch.Size([177, 3])


In [108]:
class Neural_Network(nn.Module):
    
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = x.size()[1]
        self.outputSize = y.size()[1]
        self.hiddenSize = 20
        
        # weights with + 1 for bias nodes
        self.W1 = torch.randn(self.inputSize + 1, self.hiddenSize)
        self.W2 = torch.randn(self.hiddenSize + 1, self.outputSize) 
        
    def forward(self, X):
        #print(X.size())
        #print(self.W1.size())
        X_w_bias = torch.cat((X, torch.ones(x.size()[0], 1)), 1)
        self.z = torch.matmul(X_w_bias, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        z2_w_bias = torch.cat((self.z2, torch.ones(self.z2.size()[0], 1)), 1)
        self.z3 = torch.matmul(z2_w_bias, self.W2)
        o = self.sigmoid(self.z3) # final activation function            
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))

        z2_w_bias = torch.cat((self.z2, torch.ones(self.z2.size()[0], 1)), 1)

        self.z2_delta = self.z2_error * self.sigmoidPrime(z2_w_bias)
        
        X_w_bias = torch.cat((X, torch.ones(x.size()[0], 1)), 1)
        print(self.z2_delta.size(), self.o_delta.size())
        print(torch.t(X_w_bias).size())
        print(self.W1.size())
        self.W1 += torch.matmul(torch.t(X_w_bias), self.z2_delta)
        self.W2 += torch.matmul(torch.t(z2_w_bias), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self, X):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(X))
        foward_fn = self.forward(X)
        print ("Output: \n" + str(foward_fn))
        return forward_fn

In [15]:
NN = Neural_Network()
torch.set_default_tensor_type('torch.FloatTensor')


In [109]:
NN = Neural_Network()
for i in range(1000):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(x)) ** 2).detach().item()))  # mean sum squared loss
    NN.train(x, y)
NN.saveWeights(NN)

#0 Loss: 0.46039021015167236
torch.Size([177, 21]) torch.Size([177, 3])
torch.Size([14, 177])
torch.Size([14, 20])


RuntimeError: The expanded size of the tensor (20) must match the existing size (21) at non-singleton dimension 1

In [55]:
predictions = NN.forward(x)
print("------")
#print(predictions)
#print(type(predictions))
correct = 0
for i in range(len(predictions)):
    #print("prediction: " + str(predictions[i]))
    #print("actual: " + str(y[i]))
    pred = predictions[i].tolist()
    actual = y[i].tolist()
    if pred.index(max(pred)) == actual.index(max(actual)):
        correct += 1

print("accuracy: " + str(correct / len(predictions)))
    

------
accuracy: 0.576271186440678


In [24]:
x = x.float()
y = y.float()
print(x.type())
print(y.type())

torch.FloatTensor
torch.FloatTensor


In [23]:
x

tensor([[ 0.7802,  0.7748,  0.9380,  ...,  0.7896,  0.7761,  0.9056],
        [-0.3862, -0.1862, -0.3276,  ...,  0.4759, -0.1069,  0.4138],
        [ 0.3251,  0.6533,  0.5480,  ...,  0.3994,  0.4675,  0.6966],
        ...,
        [ 0.2281,  0.2047,  0.0058,  ..., -0.3099, -0.2982, -0.2865],
        [ 0.7000,  0.5850,  0.7250,  ..., -0.2200, -0.1900, -0.2000],
        [ 0.2500,  0.4107,  0.7619,  ..., -0.0060,  0.0000, -0.3333]])

In [ ]:
# need to add bias values to input and hidden layer!!!


In [56]:
NN.predict(x)

Predicted data based on trained weights: 
Input (scaled): 
tensor([[ 0.7802, -0.3862,  0.3251,  ...,  0.2281,  0.7000,  0.2500],
        [ 0.7748, -0.1862,  0.6533,  ...,  0.2047,  0.5850,  0.4107],
        [ 0.9380, -0.3276,  0.5480,  ...,  0.0058,  0.7250,  0.7619],
        ...,
        [ 0.7896,  0.4759,  0.3994,  ..., -0.3099, -0.2200, -0.0060],
        [ 0.7761, -0.1069,  0.4675,  ..., -0.2982, -0.1900,  0.0000],
        [ 0.9056,  0.4138,  0.6966,  ..., -0.2865, -0.2000, -0.3333]])
Output: 
tensor([[7.9406e-17, 2.3666e-07, 3.7494e-11],
        [5.1035e-16, 1.3248e-07, 4.3855e-11],
        [8.4645e-14, 1.6225e-07, 3.9584e-08],
        [2.6061e-17, 1.2229e-07, 3.9222e-09],
        [9.8707e-15, 2.7804e-07, 2.2305e-09],
        [6.8918e-15, 4.7084e-07, 1.3709e-09],
        [9.3432e-15, 3.2216e-07, 5.8709e-08],
        [1.7848e-16, 2.2602e-07, 4.1539e-11],
        [6.3615e-16, 2.4281e-07, 4.1475e-10],
        [6.3757e-16, 9.5969e-08, 2.5043e-11],
        [1.3418e-14, 4.1473e-07, 7.555

In [59]:
df.to_csv("/home/dom/Documents/MPhys/TheGrandTour/wine_data.csv", index=False)

In [68]:
x.size()

torch.Size([177, 13])

In [91]:
torch.cat((x, torch.ones(x.size()[0], 1)), 1).size()

torch.Size([177, 14])

In [85]:
x.size()[0]

177